# Check AirQuality Stations

Filtre air quality sites in order to obtain those with the main requisites needed to our study.                                      

- Have data from any of the following pollutants:                                                                                                                                                                                       
    NO2, NO, O3, PM10, PM2.5                                                                                                                                                                                                          
- Don't miss any year of data  

In [1]:
setwd("~/Repositories/AirQualityCOVID/")

## Funciones Utiles

In [2]:
get.miss.stations <- function(new.df, miss.df) {
    sitesAQ <- read.csv("data/Curation/sitesAQ.csv")
    
    miss.site <- levels(as.factor(miss.df[!(miss.df$site %in% new.df$site), "site"]))
    
    sitesAQ[sitesAQ$site %in% miss.site, c("site", "site_name")]
}

In [3]:
count.miss.pollutant <- function(dataframe) {
    count <- c()
    
    for (pll in levels(as.factor(dataframe$Pollutant))) {
        count <- c(count, 
                   nrow(dataframe[dataframe$Pollutant == pll, ])
                  )
        names(count)[length(count)] <- pll
    }
    count
}

## Todas las estaciones de estudio

In [4]:
info.sitesAQ <- read.csv("data/Curation/info_sitesAQ.csv",
                         stringsAsFactor=FALSE)
info.sitesAQ <- info.sitesAQ[complete.cases(info.sitesAQ), ]
head(info.sitesAQ)

,site,Pollutant,start_yr,end_yr,hv.min,mss.wk,mss.mnth,mss.yr
,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<int>
1,es2047a,pm10,2014-01-01,2018-11-24,FALSE,168,37,3
2,es2047a,no2,2014-01-01,2020-12-30,TRUE,60,12,1
3,es2047a,no,2014-01-01,2020-12-30,TRUE,60,12,1
4,es2042a,o3,2013-04-08,2020-12-30,TRUE,14,3,0
5,es2042a,no2,2018-01-01,2020-12-30,TRUE,271,61,5
6,es2042a,no,2018-01-01,2020-12-30,TRUE,271,61,5


In [5]:
print(paste("Num Estaciones: ", 
            length(levels(as.factor(info.sitesAQ$site)))
            )
     )

print(paste("Pares (Estacion, contaminante): ", 
            nrow(info.sitesAQ)
            )
     )

[1] "Num Estaciones:  67"
[1] "Pares (Estacion, contaminante):  278"


## Datos Minimos

Tienen datos de la concentracion del contaminante en el intervalo minimo de estudio

In [6]:
sites.hvmin <- info.sitesAQ[info.sitesAQ$hv.min == TRUE &
                            !is.na(info.sitesAQ$hv.min), ]
head(sites.hvmin)

,site,Pollutant,start_yr,end_yr,hv.min,mss.wk,mss.mnth,mss.yr
,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<int>
2,es2047a,no2,2014-01-01,2020-12-30,TRUE,60,12,1
3,es2047a,no,2014-01-01,2020-12-30,TRUE,60,12,1
4,es2042a,o3,2013-04-08,2020-12-30,TRUE,14,3,0
5,es2042a,no2,2018-01-01,2020-12-30,TRUE,271,61,5
6,es2042a,no,2018-01-01,2020-12-30,TRUE,271,61,5
8,es2031a,no2,2013-01-01,2020-12-30,TRUE,20,1,0


In [7]:
print(paste("Num Estaciones: ", 
            length(levels(as.factor(sites.hvmin$site)))
            )
     )

print(paste("Pares (Estacion, contaminante): ", 
            nrow(sites.hvmin)
            )
     )

[1] "Num Estaciones:  66"
[1] "Pares (Estacion, contaminante):  238"


Se muestran los valores eliminados 

In [8]:
no.sites.hvmin <- info.sitesAQ[!info.sitesAQ$hv.min, ]

stations <- get.miss.stations(sites.hvmin, no.sites.hvmin)
pollut <- count.miss.pollutant(no.sites.hvmin)

print("Estaciones Perdidas: ")
for (i in 1:nrow(stations)) {
    print(paste("    ", 
                stations[i, "site"], ": ",
                stations[i, "site_name"], sep=""))
}


for (i in 1:length(pollut)) {
    print(paste(names(pollut[i]), pollut[[i]], sep=": "))
}

[1] "Estaciones Perdidas: "
[1] "    es1138a: CORLAB 1"
[1] "no: 1"
[1] "no2: 1"
[1] "o3: 4"
[1] "pm10: 19"
[1] "pm2.5: 15"


# No Falta Anhos

In [9]:
sites.hvyr <- sites.hvmin[sites.hvmin$mss.yr <= 0, ]
head(sites.hvyr)

,site,Pollutant,start_yr,end_yr,hv.min,mss.wk,mss.mnth,mss.yr
,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<int>
4,es2042a,o3,2013-04-08,2020-12-30,TRUE,14,3,0
8,es2031a,no2,2013-01-01,2020-12-30,TRUE,20,1,0
9,es2031a,no,2013-01-01,2020-12-30,TRUE,20,1,0
11,es2030a,pm10,2013-01-03,2020-12-30,TRUE,22,2,0
16,es2028a,pm10,2013-02-15,2020-12-30,TRUE,5,1,0
17,es2028a,o3,2013-02-15,2020-12-30,TRUE,5,1,0


In [10]:
print(paste("Num Estaciones: ", 
            length(levels(as.factor(sites.hvyr$site)))
            )
     )

print(paste("Pares (Estacion, contaminante): ", 
            nrow(sites.hvyr)
            )
     )

[1] "Num Estaciones:  58"
[1] "Pares (Estacion, contaminante):  176"


In [11]:
no.sites.hvyr <- sites.hvmin[sites.hvmin$mss.yr > 0, ]

stations <- get.miss.stations(sites.hvyr, no.sites.hvyr)
pollut <- count.miss.pollutant(no.sites.hvyr)

print("Estaciones Perdidas: ")
for (i in 1:nrow(stations)) {
    print(paste("    ", 
                stations[i, "site"], ": ",
                stations[i, "site_name"], sep=""))
}


for (i in 1:length(pollut)) {
    print(paste(names(pollut[i]), pollut[[i]], sep=": "))
}

[1] "Estaciones Perdidas: "
[1] "    es0041a: DIRECCI\303\223N DE SALUD"
[1] "    es0110a: ERANDIO"
[1] "    es1239a: VAL\303\210NCIA-PISTA DE SILLA"
[1] "    es1502a: AVENIDA GASTEIZ"
[1] "    es1836a: LAS FUENTES"
[1] "    es1837a: CENTRO"
[1] "    es1970a: VAL\303\210NCIA - BULEVARD SUD"
[1] "    es2047a: AVDA. AL-NASIR"
[1] "no: 11"
[1] "no2: 12"
[1] "o3: 12"
[1] "pm10: 11"
[1] "pm2.5: 16"


In [16]:
info.sitesAQ[info.sitesAQ$site == "es0110a",]

,site,Pollutant,start_yr,end_yr,hv.min,mss.wk,mss.mnth,mss.yr
,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<int>
272,es0110a,pm2.5,2014-04-30,2020-12-30,TRUE,68,15,1
273,es0110a,pm10,2014-04-30,2020-12-30,TRUE,68,15,1
274,es0110a,no2,2014-01-01,2020-12-30,TRUE,65,15,1
275,es0110a,no,2014-01-01,2020-12-30,TRUE,65,15,1
